### Part1: Define Functions

In [1]:
# Code to get bond Prices

import numpy as np

def gamma(kappa, sigma, c2):
    """Calculate the gamma term in the CIR functions."""
    return np.sqrt(kappa**2 + 2 * sigma**2 * c2)

def A_CIR(t, T, vartheta, sigma, kappa, c2):
    """Calculate the A term for the CIR model."""
    gamma_val = gamma(kappa, sigma, c2)
    term1 = vartheta * kappa * sigma**-2 * (kappa + gamma_val) * (T - t)
    term2 = 2 * vartheta * kappa * sigma**-2 * np.log(1 + (kappa + gamma_val) / (2 * gamma_val) * (np.exp(gamma_val * (T - t)) - 1))
    return term1 - term2

def B_CIR(t, T, vartheta, sigma, kappa, c2):
    """Calculate the B term for the CIR model."""
    gamma_val = gamma(kappa, sigma, c2)
    return 2 * c2 * (1 - np.exp(-gamma_val * (T - t))) / (kappa + gamma_val + (kappa + 2 * gamma_val) * np.exp(-gamma_val * (T - t)))

def bond_price(t, T, rd_t, alpha, beta, vartheta, sigma, kappa):
    """Calculate the bond price P(t, T) using the extended CIR model."""
    A = A_CIR(t, T, beta * vartheta + alpha, beta * sigma, kappa, 1/beta)
    B = B_CIR(t, T, beta * vartheta + alpha, beta * sigma, kappa, 1/beta)
    price = np.exp((alpha / beta) * (T - t) + A - alpha * B - beta * B * rd_t)
    return price


In [2]:
# Example bond price calculation
alpha = 0.0
beta = 1
vartheta = 0.07
sigma = 0.02
kappa = 0.9
t = 0
T = 5
rd_t = 0.01

# Calculate the bond price
price = bond_price(t, T, rd_t, alpha, beta, vartheta, sigma, kappa)
print(f"Bond price P({t}, {T}) = {price}")


Bond price P(0, 5) = 0.7528787576188439


### Part 2: Calibrate to the three markets

In [3]:
# Calibration to FX market data
import pandas as pd
import numpy as np
from scipy.optimize import minimize
for i in range




In [4]:
# Monte Carlo simuation and Derivative Valuation

def monte_carlo_option_valuation(S0, k, k_prime, N, T, delta, num_paths, num_timesteps, alpha, beta, vartheta, sigma_X, sigma_f, sigma_r, kappa):
    dt = T / num_timesteps
    sqrt_dt = np.sqrt(dt)

    # Initialize the paths for Sf_tilde and r_d as floating-point arrays
    rd = np.zeros(num_paths, dtype=np.float64)
    Sf_tilde = np.full(num_paths, S0, dtype=np.float64)

    # Simulate paths
    for _ in range(num_timesteps):
        dW_X = np.random.normal(0, sqrt_dt, (num_paths, 3))
        dW_f = np.random.normal(0, sqrt_dt, (num_paths, 3))
        dW_r = np.random.normal(0, sqrt_dt, num_paths)

        #Rates
        rd += kappa * (vartheta - rd) * dt + sigma_r * np.sqrt(alpha + beta * rd) * dW_r

        # Equity - sf_tilde stands for foreign equity in domestic currency (see model formulation)
        Sf_tilde += Sf_tilde * (rd * dt + np.dot(dW_f, sigma_f) + np.dot(dW_X, sigma_X))

    # Calculate bond prices
    P_T_plus_Delta = np.array([bond_price(T, T + delta, rd_t, alpha, beta, vartheta, sigma_X[0], kappa) for rd_t in rd])
    P_0_plus_Delta = bond_price(0, T + delta, rd[0], alpha, beta, vartheta, sigma_X[0], kappa)

    # Calculate option payoff
    payoffs = N * np.maximum(0, (Sf_tilde / S0 - k) * (k_prime - P_T_plus_Delta / P_0_plus_Delta))

    discount_factor = np.array([bond_price(0, T, rd_t, alpha, beta, vartheta, sigma_X[0], kappa) for rd_t in rd])
    option_value = np.mean(payoffs * discount_factor)

    return option_value

In [5]:
# Test the monte carlo with specified parameters

# simulation inputs
num_paths = 10000
num_timesteps = 250

# product input: not to calibrate
S0 = 100 # spot in USD at t=0
k = 1.00 # equity strike
k_prime = 1.00 # rates strike
N = 100 # notional
T = 1 # T - t, from obs to start of future
delta = 0.25 # SOFR future 3 months tenor

alpha = 0 # CALIBRATE FROM THE RATES MARKET
beta = 1 # CALIBRATE FROM RATES MARKET
vartheta = 0.05 # LONG-TERM MEAN: CALIBRATE FROM RATES MARKET
kappa = 0.9 # MEAN REVERSION SPEED: CALIBRATE FROM RATES MARKET

sigma_X = np.array([0.1, 0.1, 0.1]) # TO CALIBRATE FROM FX MARKET
sigma_f = np.array([0.2, 0.2, 0.2]) # TO CALIBRATE FROM FOREIGN EQUITY MARKET
sigma_r = 0.02

# Calculate the option value
option_value = monte_carlo_option_valuation(S0, k, k_prime, N, T, delta, num_paths, num_timesteps, alpha, beta, vartheta, sigma_X, sigma_f, sigma_r, kappa)
print(f"Estimated option value: {option_value}")

Estimated option value: 0.5242247087086697
